This demonstration is based off the companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). This notebook was generated for TensorFlow 2.6, but is current up to 2.12.0.

In [ ]:
import tensorflow as tf
import numpy as np
import time
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.metrics import AUC

print(tf.__version__)

## Demo Part I - Add TensorBoard to a model 
### The IMDB dataset
### ML problem: Binary classification, positive or negative reviews?

**Loading the IMDB dataset**

In [ ]:
from tensorflow.keras.datasets import imdb

# Load training and test data from imdb dataset
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(
    num_words=10000)

# Values of text from this dataset integer encoded; this first sample
# train_data[0]

# The label for the first sample (1 or 0)
# train_labels[0]

# The number of trainign samples 1000 total
# max([max(sequence) for sequence in train_data])

**Decoding reviews back to text**

In [ ]:
# If curious about the decoded text, set DECODE to True and run.
# Only the first sample is decoded.
DECODE = True
if DECODE:
        
    word_index = imdb.get_word_index()
    reverse_word_index = dict(
        [(value, key) for (key, value) in word_index.items()])
    decoded_review = " ".join(
        [reverse_word_index.get(i - 3, "?") for i in train_data[0]])

    # See the decoded version of the encoded words from the imdb dataset
    print(decoded_review)

### Preparing the data

**Encoding the integer sequences via multi-hot encoding**

In [ ]:

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        for j in sequence:
            results[i, j] = 1.
    return results
# create train and test datasets    
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

# See multi-hot encoded vectorrized sequence
# x_train[0]


In [ ]:
# Cast as floats for training
y_train = np.asarray(train_labels).astype("float32")
y_test = np.asarray(test_labels).astype("float32")

# Create train/validation split
x_val = x_train[:10000]
y_val = y_train[:10000]
partial_x_train = x_train[10000:]
partial_y_train = y_train[10000:]

### Building your model using TensorBoard

In [ ]:
# setup logging scheme
MODEL = "imdb-model-{}".format(int(time.time()))

# initialize callback (create logs directory first or model.fit will fail)
tbcallback = TensorBoard(log_dir = "logs/{}".format(MODEL), histogram_freq=1)
# windows?
# tbcallback = TensorBoard(log_dir = "logs\{}".format(MODEL), histogram_freq=1)

print(MODEL + " <- this will be your TensorBoard log file name.")

In [ ]:
# initialize a sequential fully connected network to address the ML problem
model = keras.Sequential([
    layers.Dense(16, activation="relu", name="L1_16_Dense_RELU"),
    layers.Dense(16, activation="relu", name="L2_16_Dense_RELU"),
    layers.Dense(1, activation="sigmoid", name="L3_1_Dense_SIG")
])

# Compile the network using your optmizier, loss, and metrics
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

# Fit your model, including TensorBoard in the callback list
model.fit(partial_x_train,
          partial_y_train, 
          validation_data=(x_val, y_val),
          epochs=20, 
          batch_size=512, 
          callbacks=[tbcallback])
            

### Review TensorBoard output to evaluate how model performance.

In [ ]:
# In console, from directory containing the "logs" directory.
# tensorboard --logdir logs

### Refit model with adjusted parameters and all training data.

In [ ]:
# Make any adjustments for your final model
model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

# added in a Precision Recall curve metric
metrics=[tf.keras.metrics.Accuracy(name='accuracy'),
        tf.keras.metrics.AUC(name='prc', curve='PR')]


# Compile the network using your optmizier, loss, and metrics
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=metrics)

# Update logging scheme
MODEL = "imdb-final-model-{}".format(int(time.time()))
tbcallback = TensorBoard(log_dir = "logs/{}".format(MODEL), histogram_freq=1)
# tbcallback = TensorBoard(log_dir = "logs\{}".format(MODEL), histogram_freq=1)

# Refit your final model
model.fit(x_train,                              # No longer partial_x_train
          y_train,                              # No longer partial_y_train
          epochs=5,                             # Changed to 5 epochs 
          batch_size=512, 
          callbacks=[tbcallback])
            

In [ ]:

# Check how accurate the model is at predicting unseen observations
results = model.evaluate(x_test, y_test)

# A tuple containing loss, accuracy
results

### Using a trained model to generate predictions on new data

In [ ]:
model.predict(x_test)

# Part 2: Comparing multiple models

## Classifying newswires: A multiclass classification example

### The Reuters dataset

In [ ]:
import tensorflow as tf
import numpy as np
import time
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.datasets import reuters

**Loading the Reuters dataset**

In [ ]:
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(
    num_words=10000)

#len(train_data)
#len(test_data)
#train_data[10]

**Decoding newswires back to text**

In [ ]:
word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_newswire = " ".join([reverse_word_index.get(i - 3, "?") for i in
    train_data[0]])

#train_labels[10]

### Preparing the data

**Encoding the input and labels & Splitting data**

In [ ]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        for j in sequence:
            results[i, j] = 1.
    return results

#Encode input
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

#Encode labels
def to_one_hot(labels, dimension=46):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results
y_train = to_one_hot(train_labels)
y_test = to_one_hot(test_labels)

# import 1-hot encoding library
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(train_labels)
y_test = to_categorical(test_labels)

#Set aside validation data
x_val = x_train[:1000]
partial_x_train = x_train[1000:]
y_val = y_train[:1000]
partial_y_train = y_train[1000:]

### Building your model

**Model definition**

In [ ]:
# Create two models, the only difference is hidden layer activations

model1 = keras.Sequential([
    layers.Dense(64, activation="tanh"),
    layers.Dense(64, activation="tanh"),
    layers.Dense(46, activation="softmax")
])

model2 = keras.Sequential([
    layers.Dense(64, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(46, activation="softmax")
])

**Compiling the model**

In [ ]:
model1.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

model2.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

**Training the model**

In [ ]:
# setup logging scheme for model1
MODEL = "mod1-reuters-{}".format(int(time.time()))
tbcallback = TensorBoard(log_dir = "logs/{}".format(MODEL), histogram_freq=1)
#tbcallback = TensorBoard(log_dir = "logs\{}".format(MODEL), histogram_freq=1)

history1 = model1.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    callbacks=[tbcallback])

MODEL = "mod2-reuters-{}".format(int(time.time()))
tbcallback = TensorBoard(log_dir = "logs/{}".format(MODEL), histogram_freq=1)
#tbcallback = TensorBoard(log_dir = "logs\{}".format(MODEL), histogram_freq=1)

history2 = model2.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    callbacks=[tbcallback])

**Retraining a model from scratch**

In [ ]:
MODEL = "final-reuters-{}".format(int(time.time()))
tbcallback = TensorBoard(log_dir = "logs/{}".format(MODEL), histogram_freq=1)
#tbcallback = TensorBoard(log_dir = "logs\{}".format(MODEL), histogram_freq=1)

model = keras.Sequential([
  layers.Dense(64, activation="relu"),
  layers.Dense(64, activation="relu"),
  layers.Dense(46, activation="softmax")
])
model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

model.fit(x_train,
          y_train,
          epochs=20,
          batch_size=512,
          callbacks=[tbcallback])
          
results = model.evaluate(x_test, y_test)


In [ ]:
results

In [ ]:
import copy
test_labels_copy = copy.copy(test_labels)
np.random.shuffle(test_labels_copy)
hits_array = np.array(test_labels) == np.array(test_labels_copy)
hits_array.mean()

### Generating predictions on new data

In [ ]:
predictions = model.predict(x_test)

In [ ]:
predictions[0].shape

In [ ]:
np.sum(predictions[0])

In [ ]:
np.argmax(predictions[0])